In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import tensorflow as tf

In [2]:
df_aqi = pd.read_csv('/Users/nikmag/Desktop/USC/Research/SSI/processed_files/processed_los_angeles_aqi_14_months.csv')
df_aqi.index = df_aqi['timestamp'].apply(pd.Timestamp)
del df_aqi['timestamp']
#df_aqi = df_aqi.loc[:,[i for i in df_aqi.columns.tolist() if i.startswith('aqi')]]

In [3]:
df_meo = pd.read_csv('/Users/nikmag/Desktop/USC/Research/SSI/processed_files/processed_los_angeles_weather_14_months.csv')
df_meo.index = df_meo['timestamp'].apply(pd.Timestamp)
del df_meo['timestamp']

In [4]:
sensors = ['W San Gabriel Vly',
 'San Gabriel Mts',
 'SW San Bernardino',
 'Southeast LA CO',
 'South Coastal LA',
 'Central LA CO',
 'Santa Clarita Vly',
 'W San Fernando Vly',
 'E San Gabriel V-2',
]

In [5]:
aqi_cols = [col for col in df_aqi.columns if col.endswith(('Antelope Vly','SW Coastal LA','NW Coastal LA','E San Fernando Vly'))]

In [6]:
meo_cols = [col for col in df_meo.columns if col.endswith(('Antelope Vly','SW Coastal LA','NW Coastal LA','E San Fernando Vly'))]

In [7]:
df_aqi.drop(columns=aqi_cols,axis=1,inplace=True)
df_meo.drop(columns=meo_cols,axis=1,inplace=True)

In [8]:
df = pd.merge(df_aqi,df_meo,how='left',left_index=True,right_index=True)

In [9]:
df.replace(0.0,np.NaN,inplace=True)

In [10]:
def masked_mse_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.square(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)


def masked_mae_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.abs(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)


def masked_rmse_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    return tf.sqrt(masked_mse_tf(preds=preds, labels=labels, null_val=null_val))


def masked_rmse_np(preds, labels, null_val=np.nan):
    return np.sqrt(masked_mse_np(preds=preds, labels=labels, null_val=null_val))


def masked_mse_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        rmse = np.square(np.subtract(preds, labels)).astype('float32')
        rmse = np.nan_to_num(rmse * mask)
        return np.mean(rmse)


def masked_mae_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(preds, labels)).astype('float32')
        mae = np.nan_to_num(mae * mask)
        return np.mean(mae)


def masked_mape_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mape = np.abs(np.divide(np.subtract(preds, labels).astype('float32'), labels))
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape)


# Builds loss function.
def masked_mse_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        return masked_mse_tf(preds=preds, labels=labels, null_val=null_val)

    return loss


def masked_rmse_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        return masked_rmse_tf(preds=preds, labels=labels, null_val=null_val)

    return loss


def masked_mae_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        mae = masked_mae_tf(preds=preds, labels=labels, null_val=null_val)
        return mae

    return loss


def calculate_metrics(df_pred, df_test, null_val):
    """
    Calculate the MAE, MAPE, RMSE
    :param df_pred:
    :param df_test:
    :param null_val:
    :return:
    """
    mape = masked_mape_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    mae = masked_mae_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    rmse = masked_rmse_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    return [mae, mape, rmse]

## 24 Lagged Variables

Input - AQI(t), AQI(t-1),..., AQI(t-24) + Weather vars (t,....,t-24)

Output - AQI(t+1), AQI(t+6), AQI(t+12), AQI(t+18), AQI(t+24) (Multi-output prediction)

### Linear Regression

In [11]:
pred = {k:[] for k in [1,6,12,18,24]}
gt = {k:[] for k in [1,6,12,18,24]}

for sensor in sensors:
    
    start = time.time()
    
    df_temp = df.loc[:,[i for i in df.columns.tolist() if i.endswith(sensor)]]
    
    for col in ['aqi','pressure','wind_speed','cloud_cover','visibility','wind_bearing','humidity','temperature']: 
        for j in range(1,24):
            df_temp['p{}_{}'.format(j,col)] = df_temp[col + '_%s' % sensor]
            df_temp['p{}_{}'.format(j,col)] = df_temp['p{}_{}'.format(j,col)].shift(j)

    for j in [1,6,12,18,24]:
        df_temp['hz{}'.format(j)] = df_temp['aqi_%s' % sensor]
        df_temp['hz{}'.format(j)] = df_temp['hz{}'.format(j)].shift(-j)
    
    y_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')]
    x_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')==False]
    
    X = df_temp.loc[:,x_cols]    
    y = df_temp.loc[:,y_cols]
    
    X_train = X.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    y_train = y.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    X_test = X.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    y_test = y.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    
    #nik
    train = pd.concat([X_train,y_train],axis=1)
    train.dropna(axis=0,how='any',inplace=True)
    X_train = train.loc[:,x_cols]
    y_train = train.loc[:,y_cols]
    
    #nik
    X_test.fillna(X_test.mean(),inplace=True)

    model = MultiTaskLassoCV(alphas=[0.1,1,10,100],cv=3)
    model.fit(X_train,y_train)
    
    predictions = model.predict(X_test)
    actual = np.array(y_test)
    
    j=0
    for i in [1,6,12,18,24]:
        pred[i].append(predictions[:,j].tolist())
        gt[i].append(actual[:,j].tolist())
        j = j + 1
        
    end = time.time()
    
    print(sensor+'='+str(np.round(end-start,2)))

/Users/nikmag/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/nikmag/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


W San Gabriel Vly=71.64
San Gabriel Mts=94.32
SW San Bernardino=111.7
Southeast LA CO=111.85
South Coastal LA=86.47
Central LA CO=30.51
Santa Clarita Vly=27.42
W San Fernando Vly=30.49
E San Gabriel V-2=29.69


In [12]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for i in gt.keys():
    res = calculate_metrics(pd.DataFrame(np.array(pred[i]).T),pd.DataFrame(np.array(gt[i]).T),np.nan)
    result_mape[i] = res[1]
    result_mae[i]= res[0]
    result_rmse[i]= res[2]

final_dict = {k:[] for k in [1,6,12,18,24]}
for i in [1,6,12,18,24]:
    final_dict[i].append(np.round(result_mae[i],2))
    final_dict[i].append(np.round(result_rmse[i],2))
    final_dict[i].append(np.round(result_mape[i],2))

for i in [1,6,12,18,24]:
    for j in final_dict[i]:
        print(j)

6.04
8.94
0.27
14.33
18.7
0.6
16.16
20.83
0.71
17.06
21.95
0.77
17.38
22.43
0.79


### VAR

In [11]:
pred = {k:[] for k in [1,6,12,18,24]}
gt = {k:[] for k in [1,6,12,18,24]}

for sensor in sensors:
    
    start = time.time()
    
    df_temp = df.loc[:,[i for i in df.columns.tolist()]]
    
    for k in sensors:
        for col in ['aqi','pressure','wind_speed','cloud_cover','visibility','wind_bearing','humidity','temperature']:
            for j in range(1,24):
                df_temp['p{}_{}_{}'.format(j,k,col)] = df_temp[col + '_%s' % k]
                df_temp['p{}_{}_{}'.format(j,k,col)] = df_temp['p{}_{}_{}'.format(j,k,col)].shift(j)
    
    for j in [1,6,12,18,24]:
        df_temp['hz{}'.format(j)] = df_temp['aqi_%s' % sensor]
        df_temp['hz{}'.format(j)] = df_temp['hz{}'.format(j)].shift(-j)
    
    y_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')]
    x_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')==False]
    
    X = df_temp.loc[:,x_cols]    
    y = df_temp.loc[:,y_cols]
    
    X_train = X.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    y_train = y.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    X_test = X.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    y_test = y.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    
    #nik
    train = pd.concat([X_train,y_train],axis=1)
    train.dropna(axis=0,how='any',inplace=True)
    X_train = train.loc[:,x_cols]
    y_train = train.loc[:,y_cols]
    
    #nik
    X_test.fillna(X_test.mean(),inplace=True)

    model = MultiTaskLassoCV(alphas=[1,10,100],cv=3)
    model.fit(X_train,y_train)
    
    predictions = model.predict(X_test)
    actual = np.array(y_test)
    
    j=0
    for i in [1,6,12,18,24]:
        pred[i].append(predictions[:,j].tolist())
        gt[i].append(actual[:,j].tolist())
        j = j + 1
        
    end = time.time()
    
    print(sensor+'='+str(np.round(end-start,2)))

/Users/nikmag/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/nikmag/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


W San Gabriel Vly=159.11
San Gabriel Mts=178.67


/Users/nikmag/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:1778: ConvergenceWarning: Objective did not converge, you might want to increase the number of iterations
  ConvergenceWarning)


SW San Bernardino=187.02
Southeast LA CO=186.31
South Coastal LA=165.83
Central LA CO=166.8
Santa Clarita Vly=220.22
W San Fernando Vly=299.73
E San Gabriel V-2=288.48


In [32]:
for i in pred.keys():
    pred_csv = pd.DataFrame(np.array(pred[i])).T
    pred_csv.columns = ['pred_{}'.format(sensor) for sensor in sensors]
    pred_csv.index = X_test.index
    gt_csv = pd.DataFrame(np.array(gt[i])).T
    gt_csv.columns = ['actual_{}'.format(sensor) for sensor in sensors]
    gt_csv.index = X_test.index
    main_csv = pd.concat([pred_csv,gt_csv],axis=1)
    main_csv.to_csv('/Users/nikmag/Desktop/var_horizon_{}_output.csv'.format(i))

In [31]:
pred.keys()

[24, 1, 18, 12, 6]

In [14]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for i in gt.keys():
    res = calculate_metrics(pd.DataFrame(np.array(pred[i]).T),pd.DataFrame(np.array(gt[i]).T),np.nan)
    result_mape[i] = res[1]
    result_mae[i]= res[0]
    result_rmse[i]= res[2]

final_dict = {k:[] for k in [1,6,12,18,24]}
for i in [1,6,12,18,24]:
    final_dict[i].append(np.round(result_mae[i],2))
    final_dict[i].append(np.round(result_rmse[i],2))
    final_dict[i].append(np.round(result_mape[i],2))

for i in [1,6,12,18,24]:
    for j in final_dict[i]:
        print(j)

5.93
8.72
0.27
13.81
18.01
0.58
15.58
20.2
0.69
16.83
21.56
0.78
17.31
22.23
0.81


### GBM

In [15]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for sensor in sensors:
    
    start = time.time()
    
    df_temp = df.loc[:,[i for i in df.columns.tolist() if i.endswith(sensor)]]
    
    for col in ['aqi','pressure','wind_speed','cloud_cover','visibility','wind_bearing','humidity','temperature']: 
        for j in range(1,24):
            df_temp['p{}_{}'.format(j,col)] = df_temp[col + '_%s' % sensor]
            df_temp['p{}_{}'.format(j,col)] = df_temp['p{}_{}'.format(j,col)].shift(j)

    for j in [1,6,12,18,24]:
        df_temp['hz{}'.format(j)] = df_temp['aqi_%s' % sensor]
        df_temp['hz{}'.format(j)] = df_temp['hz{}'.format(j)].shift(-j)
    
    y_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')]
    x_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')==False]
    
    X = df_temp.loc[:,x_cols]    
    y = df_temp.loc[:,y_cols]
    
    X_train = X.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    y_train = y.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    X_test = X.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    y_test = y.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    
    #nik
    train = pd.concat([X_train,y_train],axis=1)
    train.dropna(axis=0,how='any',inplace=True)
    X_train = train.loc[:,x_cols]
    y_train = train.loc[:,y_cols]
    
    #nik
    X_test.fillna(X_test.mean(),inplace=True)

    gbm = GradientBoostingRegressor(learning_rate=0.05,max_features=0.6,max_leaf_nodes=31,n_estimators=200)
    model = MultiOutputRegressor(gbm,-1)
    model.fit(X_train,y_train)
    
    predictions = model.predict(X_test)
    actual = np.array(y_test)
    
    j=0
    for i in [1,6,12,18,24]:
        pred[i].append(predictions[:,j].tolist())
        gt[i].append(actual[:,j].tolist())
        j = j + 1
        
    end = time.time()
    
    print(sensor+'='+str(np.round(end-start,2)))

W San Gabriel Vly=35.21
San Gabriel Mts=36.75
SW San Bernardino=34.35
Southeast LA CO=37.18
South Coastal LA=34.38
Central LA CO=36.28
Santa Clarita Vly=33.28
W San Fernando Vly=37.75
E San Gabriel V-2=33.91


In [16]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for i in gt.keys():
    res = calculate_metrics(pd.DataFrame(np.array(pred[i]).T),pd.DataFrame(np.array(gt[i]).T),np.nan)
    result_mape[i] = res[1]
    result_mae[i]= res[0]
    result_rmse[i]= res[2]

final_dict = {k:[] for k in [1,6,12,18,24]}
for i in [1,6,12,18,24]:
    final_dict[i].append(np.round(result_mae[i],2))
    final_dict[i].append(np.round(result_rmse[i],2))
    final_dict[i].append(np.round(result_mape[i],2))

for i in [1,6,12,18,24]:
    for j in final_dict[i]:
        print(j)

6.01
8.86
0.28
13.63
17.87
0.59
15.68
20.4
0.7
16.99
21.91
0.78
17.6
22.68
0.81


### RF

In [18]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for sensor in sensors:
    
    start = time.time()
    
    df_temp = df.loc[:,[i for i in df.columns.tolist() if i.endswith(sensor)]]
    
    for col in ['aqi','pressure','wind_speed','cloud_cover','visibility','wind_bearing','humidity','temperature']: 
        for j in range(1,24):
            df_temp['p{}_{}'.format(j,col)] = df_temp[col + '_%s' % sensor]
            df_temp['p{}_{}'.format(j,col)] = df_temp['p{}_{}'.format(j,col)].shift(j)

    for j in [1,6,12,18,24]:
        df_temp['hz{}'.format(j)] = df_temp['aqi_%s' % sensor]
        df_temp['hz{}'.format(j)] = df_temp['hz{}'.format(j)].shift(-j)
    
    y_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')]
    x_cols = [i for i in df_temp.columns.tolist() if i.startswith('hz')==False]
    
    X = df_temp.loc[:,x_cols]    
    y = df_temp.loc[:,y_cols]
    
    X_train = X.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    y_train = y.loc[:pd.Timestamp('2017-12-31 23:00:00'),:]
    X_test = X.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    y_test = y.loc[pd.Timestamp('2018-01-01 00:00:00'):,:]
    
    #nik
    train = pd.concat([X_train,y_train],axis=1)
    train.dropna(axis=0,how='any',inplace=True)
    X_train = train.loc[:,x_cols]
    y_train = train.loc[:,y_cols]
    
    #nik
    X_test.fillna(X_test.mean(),inplace=True)

    model = RandomForestRegressor(n_estimators=600)
    model.fit(X_train,y_train)
    
    predictions = model.predict(X_test)
    actual = np.array(y_test)
    
    j=0
    for i in [1,6,12,18,24]:
        pred[i].append(predictions[:,j].tolist())
        gt[i].append(actual[:,j].tolist())
        j = j + 1
        
    end = time.time()
    
    print(sensor+'='+str(np.round(end-start,2)))

W San Gabriel Vly=402.59
San Gabriel Mts=471.79
SW San Bernardino=409.71
Southeast LA CO=471.17
South Coastal LA=428.93
Central LA CO=457.54
Santa Clarita Vly=396.01
W San Fernando Vly=487.0
E San Gabriel V-2=373.67


In [19]:
result_mape = {k:[] for k in [1,6,12,18,24]}
result_mae = {k:[] for k in [1,6,12,18,24]}
result_rmse = {k:[] for k in [1,6,12,18,24]}

for i in gt.keys():
    res = calculate_metrics(pd.DataFrame(np.array(pred[i]).T),pd.DataFrame(np.array(gt[i]).T),np.nan)
    result_mape[i] = res[1]
    result_mae[i]= res[0]
    result_rmse[i]= res[2]

final_dict = {k:[] for k in [1,6,12,18,24]}
for i in [1,6,12,18,24]:
    final_dict[i].append(np.round(result_mae[i],2))
    final_dict[i].append(np.round(result_rmse[i],2))
    final_dict[i].append(np.round(result_mape[i],2))

for i in [1,6,12,18,24]:
    for j in final_dict[i]:
        print(j)

6.33
9.29
0.29
13.86
18.1
0.59
15.92
20.66
0.69
17.27
22.27
0.77
17.82
23.01
0.79
